In [ ]:
# SAHNIK0 GOOGLE EARTH ENGINE SATELLITE MONITORING SYSTEM
# COMPLETE FIXED VERSION WITH WATER BODY ANALYSIS
# Project ID: earth-engine-464312
# Current Time: 2025-07-01 12:18:49

!pip install schedule earthengine-api ipywidgets python-dotenv
import os
import time
import smtplib
import logging
import json
import threading
import schedule
import numpy as np
import math
from datetime import datetime, timedelta
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
from email.mime.text import MIMEText
from email.mime.image import MIMEImage
from abc import ABC, abstractmethod
from typing import Dict, List, Any, Optional, Tuple
import io

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Essential imports
from IPython.display import display, HTML, clear_output
try:
    from ipywidgets import widgets
    import ee
    import requests
    from PIL import Image, ImageDraw, ImageFont
    print("✅ All dependencies loaded")
except ImportError as e:
    print(f"⚠️ Import error: {e}")

# ---- CONFIGURATION FROM ENVIRONMENT VARIABLES ----
PROJECT_ID = os.getenv('PROJECT_ID', 'earth-engine-464312')
EMAIL_HOST = os.getenv('EMAIL_HOST', 'smtp.gmail.com')
EMAIL_PORT = int(os.getenv('EMAIL_PORT', '587'))
EMAIL_USER = os.getenv('EMAIL_USER')
EMAIL_PASS = os.getenv('EMAIL_PASS')
EMAIL_COOLDOWN = int(os.getenv('EMAIL_COOLDOWN', '5'))
MONITORING_INTERVAL = int(os.getenv('MONITORING_INTERVAL', '300'))
RESULTS_DIR = "gee_results"
LOG_FILE = os.path.join(RESULTS_DIR, "monitoring_log.txt")
ALERT_HISTORY_FILE = os.path.join(RESULTS_DIR, "alert_history.json")

CURRENT_TIME = "2025-07-01 12:18:49"
CURRENT_USER = os.getenv('CURRENT_USER', 'Sahnik0')

# Validate required environment variables
if not EMAIL_USER or not EMAIL_PASS:
    print("⚠️ WARNING: EMAIL_USER or EMAIL_PASS not found in environment variables!")
    print("Please check your .env file and ensure these variables are set.")

# Detection thresholds from environment variables
DETECTION_THRESHOLDS = {
    'ndvi_change': float(os.getenv('NDVI_CHANGE_THRESHOLD', '0.15')),
    'mndwi_change': float(os.getenv('MNDWI_CHANGE_THRESHOLD', '0.12')),
    'ndbi_change': float(os.getenv('NDBI_CHANGE_THRESHOLD', '0.10')),
    'min_area_pixels': int(os.getenv('MIN_AREA_PIXELS', '50')),
    'cloud_cover_limit': int(os.getenv('CLOUD_COVER_LIMIT', '25')),
    'confidence_threshold': float(os.getenv('CONFIDENCE_THRESHOLD', '0.3'))
}

# Global monitoring control and date tracking
monitoring_active = False
monitoring_thread = None
last_analysis_date = None
initial_start_date = None

# Setup directories
os.makedirs(RESULTS_DIR, exist_ok=True)
logging.basicConfig(filename=LOG_FILE, level=logging.INFO, format="%(asctime)s|%(levelname)s|%(message)s")

print(f"✅ Configuration loaded from environment variables")
print(f"📧 Email User: {EMAIL_USER[:5]}***@{EMAIL_USER.split('@')[1] if EMAIL_USER else 'Not Set'}")
print(f"🌍 Project ID: {PROJECT_ID}")

# ---- ALERT HISTORY MANAGEMENT ----
def load_alert_history():
    """Load alert history from file"""
    try:
        if os.path.exists(ALERT_HISTORY_FILE):
            with open(ALERT_HISTORY_FILE, 'r') as f:
                return json.load(f)
        return {}
    except Exception:
        return {}

def save_alert_history(history):
    """Save alert history to file"""
    try:
        with open(ALERT_HISTORY_FILE, 'w') as f:
            json.dump(history, f, indent=2)
    except Exception as e:
        print(f"❌ Failed to save alert history: {e}")

def create_alert_key(change_type, location, date, area_hectares):
    """Create unique key for alert tracking"""
    return f"{change_type}_{location}_{date}_{int(area_hectares)}"

def is_duplicate_alert(change_type, location, date, area_hectares):
    """Check if this alert was already sent"""
    history = load_alert_history()
    alert_key = create_alert_key(change_type, location, date, area_hectares)
    return alert_key in history

def mark_alert_sent(change_type, location, date, area_hectares):
    """Mark alert as sent in history"""
    history = load_alert_history()
    alert_key = create_alert_key(change_type, location, date, area_hectares)
    history[alert_key] = {
        'sent_time': CURRENT_TIME,
        'change_type': change_type,
        'location': location,
        'date': date,
        'area_hectares': area_hectares
    }
    save_alert_history(history)

# ---- COORDINATE VALIDATION ----
def validate_coordinates(latitude, longitude):
    """Validate if coordinates are reasonable"""
    if not (-90 <= latitude <= 90):
        return False, f"Latitude {latitude} out of range (-90 to 90)"
    if not (-180 <= longitude <= 180):
        return False, f"Longitude {longitude} out of range (-180 to 180)"
    return True, "Valid coordinates"

def create_aoi_from_coordinates(latitude, longitude, buffer_km):
    """Create AOI from coordinates with buffer"""
    try:
        is_valid, message = validate_coordinates(latitude, longitude)
        if not is_valid:
            raise ValueError(message)

        lat_buffer = buffer_km / 111.0
        lon_buffer = buffer_km / (111.0 * abs(math.cos(math.radians(latitude))))

        coords = [
            [longitude - lon_buffer, latitude - lat_buffer],
            [longitude + lon_buffer, latitude - lat_buffer],
            [longitude + lon_buffer, latitude + lat_buffer],
            [longitude - lon_buffer, latitude + lat_buffer],
            [longitude - lon_buffer, latitude - lat_buffer]
        ]

        polygon = ee.Geometry.Polygon([coords])

        return {
            'name': f"Location_{latitude:.4f}N_{longitude:.4f}E",
            'polygon': polygon,
            'center_lat': latitude,
            'center_lon': longitude,
            'buffer_km': buffer_km
        }

    except Exception as e:
        print(f"❌ AOI creation error: {e}")
        raise

# ---- AUTHENTICATION ----
def authenticate_gee():
    """Authenticate Google Earth Engine"""
    try:
        ee.Authenticate()
        ee.Initialize(project=PROJECT_ID)

        test_collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').limit(1)
        test_size = test_collection.size().getInfo()

        if test_size >= 0:
            print("✅ GEE authentication successful")
            return True
        else:
            print("❌ GEE authentication failed")
            return False

    except Exception as e:
        print(f"❌ GEE authentication error: {e}")
        return False

# ---- DATA PROCESSING ----
def compute_indices(img):
    """Compute spectral indices"""
    try:
        img = img.multiply(0.0001)
        ndvi = img.normalizedDifference(['B8', 'B4']).clamp(-1, 1).rename('NDVI')
        mndwi = img.normalizedDifference(['B3', 'B11']).clamp(-1, 1).rename('MNDWI')
        ndbi = img.normalizedDifference(['B11', 'B8']).clamp(-1, 1).rename('NDBI')

        return {'NDVI': ndvi, 'MNDWI': mndwi, 'NDBI': ndbi}
    except Exception as e:
        print(f"❌ Index calculation error: {e}")
        return {}

def apply_cloud_mask(image):
    """Apply cloud mask to Sentinel-2 image"""
    try:
        scl = image.select('SCL')
        cloud_mask = scl.eq(8).Or(scl.eq(9)).Or(scl.eq(3)).Not()
        return image.updateMask(cloud_mask)
    except Exception as e:
        print(f"❌ Cloud masking error: {e}")
        return image

def get_satellite_data(aoi, start_date=None):
    """Get satellite data with smart date progression"""
    global last_analysis_date, initial_start_date

    try:
        current_date = datetime.strptime(CURRENT_TIME[:10], '%Y-%m-%d')

        # First run - use provided start date
        if last_analysis_date is None:
            if start_date:
                initial_start_date = start_date
                current_start = start_date
                current_end = current_date.strftime('%Y-%m-%d')
                baseline_start = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=365)).strftime('%Y-%m-%d')
                baseline_end = (datetime.strptime(start_date, '%Y-%m-%d') - timedelta(days=335)).strftime('%Y-%m-%d')
            else:
                initial_start_date = (current_date - timedelta(days=30)).strftime('%Y-%m-%d')
                current_start = initial_start_date
                current_end = current_date.strftime('%Y-%m-%d')
                baseline_start = (current_date - timedelta(days=395)).strftime('%Y-%m-%d')
                baseline_end = (current_date - timedelta(days=365)).strftime('%Y-%m-%d')

            last_analysis_date = current_end
        else:
            # Subsequent runs - analyze only new period since last analysis
            baseline_start = last_analysis_date
            baseline_end = last_analysis_date
            current_start = last_analysis_date
            current_end = current_date.strftime('%Y-%m-%d')

            # Update last analysis date
            last_analysis_date = current_end

        print(f"Analysis Period: {current_start} to {current_end}")

        collection = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterBounds(aoi['polygon'])

        baseline_collection = (collection
                             .filterDate(baseline_start, baseline_end)
                             .map(apply_cloud_mask)
                             .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', DETECTION_THRESHOLDS['cloud_cover_limit'])))

        current_collection = (collection
                            .filterDate(current_start, current_end)
                            .map(apply_cloud_mask)
                            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', DETECTION_THRESHOLDS['cloud_cover_limit'])))

        baseline_image = baseline_collection.median()
        current_image = current_collection.median()

        try:
            latest_date = ee.Date(current_collection.first().get('system:time_start')).format('YYYY-MM-dd').getInfo()
        except:
            latest_date = current_end

        return {
            'before_img': baseline_image,
            'after_img': current_image,
            'before_date': baseline_start if last_analysis_date != current_end else f"Baseline ({baseline_start})",
            'after_date': latest_date,
            'periods': {
                'baseline': f"{baseline_start} to {baseline_end}",
                'current': f"{current_start} to {current_end}"
            }
        }

    except Exception as e:
        print(f"❌ Data retrieval error: {e}")
        raise

# ---- DETECTION ALGORITHMS ----
class ChangeDetector:
    def __init__(self, name, threshold_key):
        self.name = name
        self.threshold = DETECTION_THRESHOLDS[threshold_key]

    def detect_change(self, before_idx, after_idx, change_type):
        """Generic change detection with all change types"""
        try:
            if change_type == 'deforestation':
                diff = before_idx['NDVI'].subtract(after_idx['NDVI'])
                initial_mask = before_idx['NDVI'].gt(0.3)
                change_mask = diff.gt(self.threshold).And(initial_mask)

            elif change_type == 'water_loss':
                diff = before_idx['MNDWI'].subtract(after_idx['MNDWI'])
                initial_mask = before_idx['MNDWI'].gt(0.2)
                change_mask = diff.gt(self.threshold).And(initial_mask)

            elif change_type == 'water_gain':
                diff = after_idx['MNDWI'].subtract(before_idx['MNDWI'])
                initial_mask = before_idx['MNDWI'].lt(0.2)
                change_mask = diff.gt(self.threshold).And(initial_mask)

            elif change_type == 'urban_growth':
                diff = after_idx['NDBI'].subtract(before_idx['NDBI'])
                initial_mask = before_idx['NDBI'].lt(0.1)
                change_mask = diff.gt(self.threshold).And(initial_mask)

            elif change_type == 'vegetation_growth':
                diff = after_idx['NDVI'].subtract(before_idx['NDVI'])
                initial_mask = before_idx['NDVI'].lt(0.3)
                change_mask = diff.gt(self.threshold).And(initial_mask)

            else:
                return None, None

            change_mask = change_mask.focal_mode(radius=1, kernelType='square')
            return change_mask.selfMask(), diff

        except Exception as e:
            print(f"❌ {self.name} detection error: {e}")
            return None, None

# ---- IMAGE EXPORT ----
def export_images(aoi, before_img, after_img, change_mask, change_type, meta):
    """Export proof images"""
    try:
        vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        image_paths = {}

        for img_type, img, date in [('before', before_img, meta['before_date']),
                                   ('after', after_img, meta['after_date'])]:
            try:
                url = img.select(['B4', 'B3', 'B2']).visualize(**vis_params).getThumbURL({
                    'region': aoi['polygon'],
                    'dimensions': 512,
                    'format': 'png'
                })

                response = requests.get(url, timeout=30)
                if response.status_code == 200:
                    path = os.path.join(RESULTS_DIR, f"{change_type}_{img_type}_{timestamp}.png")
                    with open(path, 'wb') as f:
                        f.write(response.content)
                    image_paths[f'{img_type}_image'] = path

            except Exception as e:
                print(f"❌ Failed to export {img_type} image: {e}")

        if change_mask is not None:
            try:
                change_vis = change_mask.visualize(palette=['#FF0000'], opacity=0.8)
                overlay_img = after_img.select(['B4', 'B3', 'B2']).visualize(**vis_params).blend(change_vis)
                overlay_url = overlay_img.getThumbURL({
                    'region': aoi['polygon'],
                    'dimensions': 512,
                    'format': 'png'
                })

                response = requests.get(overlay_url, timeout=30)
                if response.status_code == 200:
                    path = os.path.join(RESULTS_DIR, f"{change_type}_overlay_{timestamp}.png")
                    with open(path, 'wb') as f:
                        f.write(response.content)
                    image_paths['overlay_image'] = path

            except Exception as e:
                print(f"❌ Failed to export overlay: {e}")

        return image_paths

    except Exception as e:
        print(f"❌ Image export error: {e}")
        return {}

# ---- EMAIL SYSTEM ----
def send_email_alert(recipient_email, subject, body, proof_images=None):
    """Send email alert"""
    try:
        msg = MIMEMultipart()
        msg['From'] = f"Sahnik0 Monitoring <{EMAIL_USER}>"
        msg['To'] = recipient_email
        msg['Subject'] = subject

        html_body = f"""
        <html>
        <body style="font-family: Arial, sans-serif;">
        <h2 style="color: #333;">{subject}</h2>
        <div style="background: #f9f9f9; padding: 15px; border-radius: 5px;">
        <pre style="white-space: pre-wrap;">{body}</pre>
        </div>
        <p style="color: #666;">Generated: {CURRENT_TIME} UTC</p>
        </body>
        </html>
        """

        msg.attach(MIMEText(html_body, 'html'))

        if proof_images:
            for img_type, img_path in proof_images.items():
                if img_path and os.path.exists(img_path):
                    try:
                        with open(img_path, 'rb') as f:
                            img_data = f.read()
                        img = MIMEImage(img_data)
                        img.add_header('Content-Disposition', f'attachment; filename="{os.path.basename(img_path)}"')
                        msg.attach(img)
                    except Exception as e:
                        print(f"❌ Image attachment error: {e}")

        server = smtplib.SMTP(EMAIL_HOST, EMAIL_PORT)
        server.starttls()
        server.login(EMAIL_USER, EMAIL_PASS)
        server.send_message(msg)
        server.quit()

        print(f"✅ Email sent: {subject}")
        return True

    except Exception as e:
        print(f"❌ Email error: {e}")
        return False

# ---- MONITORING SYSTEM ----
class MonitoringSystem:
    def __init__(self):
        self.detectors = {
            'deforestation': ChangeDetector('deforestation', 'ndvi_change'),
            'water_loss': ChangeDetector('water_loss', 'mndwi_change'),
            'water_gain': ChangeDetector('water_gain', 'mndwi_change'),
            'urban_growth': ChangeDetector('urban_growth', 'ndbi_change'),
            'vegetation_growth': ChangeDetector('vegetation_growth', 'ndvi_change')
        }
        self.monitoring_count = 0

    def run_monitoring_cycle(self, latitude, longitude, buffer_km, start_date=None, recipient_email=EMAIL_USER):
        """Run monitoring cycle with proper date progression and all change types"""
        try:
            self.monitoring_count += 1
            print(f"\n=== MONITORING CYCLE #{self.monitoring_count} ===")
            print(f"Location: {latitude:.4f}°N, {longitude:.4f}°E")

            aoi = create_aoi_from_coordinates(latitude, longitude, buffer_km)
            data = get_satellite_data(aoi, start_date)

            # Skip if no new data period
            current_period = data['periods']['current'].split(' to ')
            if len(current_period) == 2 and current_period[0] == current_period[1]:
                print("⚠️ No new data period to analyze")
                return

            before_idx = compute_indices(data['before_img'])
            after_idx = compute_indices(data['after_img'])

            if not before_idx or not after_idx:
                raise RuntimeError("Failed to compute indices")

            detections_found = 0

            # Process each detection type separately
            for detector_name, detector in self.detectors.items():
                try:
                    change_type = detector_name
                    mask, diff = detector.detect_change(before_idx, after_idx, change_type)

                    if mask is None:
                        continue

                    stats = mask.reduceRegion(
                        reducer=ee.Reducer.sum(),
                        geometry=aoi['polygon'],
                        scale=30,
                        maxPixels=1e9,
                        bestEffort=True
                    ).getInfo()

                    change_pixels = max([v for v in stats.values() if v is not None] + [0])

                    if change_pixels < DETECTION_THRESHOLDS['min_area_pixels']:
                        continue

                    confidence = min(0.95, change_pixels / 1000.0)

                    if confidence < DETECTION_THRESHOLDS['confidence_threshold']:
                        continue

                    area_hectares = round(change_pixels * 0.09, 2)
                    location_str = f"{latitude:.4f}N_{longitude:.4f}E"

                    # Check for duplicates
                    if is_duplicate_alert(change_type, location_str, data['after_date'], area_hectares):
                        print(f"⚠️ Duplicate {change_type} alert skipped")
                        continue

                    detections_found += 1

                    meta = {
                        "type": change_type,
                        "before_date": data['before_date'],
                        "after_date": data['after_date'],
                        "location": f"{latitude:.4f}°N, {longitude:.4f}°E",
                        "confidence": confidence,
                        "pixels_affected": change_pixels,
                        "area_hectares": area_hectares,
                        "cycle": self.monitoring_count
                    }

                    proof_images = export_images(aoi, data['before_img'], data['after_img'], mask, change_type, meta)

                    print(f"✅ {change_type.upper()} detected: {area_hectares} hectares, confidence: {confidence:.3f}")

                    # Create appropriate subject and emoji based on change type
                    change_emojis = {
                        'deforestation': '🌳🔥',
                        'water_loss': '💧⬇️',
                        'water_gain': '💧⬆️',
                        'urban_growth': '🏘️📈',
                        'vegetation_growth': '🌱📈'
                    }

                    emoji = change_emojis.get(change_type, '🚨')
                    subject = f"{emoji} {change_type.upper().replace('_', ' ')} DETECTED - {area_hectares} hectares"

                    body = f"""SATELLITE CHANGE DETECTION ALERT

Location: {meta['location']}
Change Type: {meta['type'].replace('_', ' ').title()}
Time Period: {meta['before_date']} → {meta['after_date']}
Area Affected: ~{meta['area_hectares']} hectares
Pixels Changed: {meta['pixels_affected']:,}
Confidence: {meta['confidence']:.3f}

This detection is based on Sentinel-2 satellite imagery analysis.
Attached images show the detected changes.

Monitoring Cycle: #{meta['cycle']}
Generated: {CURRENT_TIME} UTC
User: {CURRENT_USER}"""

                    # Send email and mark as sent
                    if send_email_alert(recipient_email, subject, body, proof_images):
                        mark_alert_sent(change_type, location_str, data['after_date'], area_hectares)
                        print(f"📧 {change_type} alert email sent")

                    time.sleep(EMAIL_COOLDOWN)

                except Exception as detection_error:
                    print(f"❌ {detector_name} processing error: {detection_error}")
                    continue

            if detections_found == 0:
                print("✅ No new changes detected in current period")

            print(f"=== CYCLE #{self.monitoring_count} COMPLETE ===")

        except Exception as e:
            print(f"❌ Monitoring cycle error: {e}")
            logging.error(f"Cycle error: {e}")

# ---- MONITORING CONTROL ----
def start_monitoring(latitude, longitude, buffer_km, start_date, recipient_email, interval_minutes):
    """Start monitoring with specified parameters"""
    global monitoring_active, monitoring_thread, MONITORING_INTERVAL

    if monitoring_active:
        print("Monitoring already active!")
        return

    MONITORING_INTERVAL = interval_minutes * 60
    monitoring_active = True
    system = MonitoringSystem()

    def monitoring_loop():
        while monitoring_active:
            try:
                system.run_monitoring_cycle(latitude, longitude, buffer_km, start_date, recipient_email)
                time.sleep(MONITORING_INTERVAL)
            except Exception as e:
                print(f"❌ Monitoring loop error: {e}")
                time.sleep(60)

    monitoring_thread = threading.Thread(target=monitoring_loop, daemon=True)
    monitoring_thread.start()

    print(f"✅ Monitoring started for {latitude:.4f}°N, {longitude:.4f}°E")

def stop_monitoring():
    """Stop monitoring"""
    global monitoring_active
    monitoring_active = False
    print("🛑 Monitoring stopped")

def reset_monitoring_dates():
    """Reset monitoring dates to start fresh"""
    global last_analysis_date, initial_start_date
    last_analysis_date = None
    initial_start_date = None
    print("🔄 Monitoring dates reset")

# ---- DASHBOARD ----
def create_dashboard():
    """Create monitoring dashboard"""
    print("Initializing Complete Dashboard...")

    if widgets is None:
        print("❌ Widgets not available")
        return

    # Input widgets
    latitude_widget = widgets.FloatText(value=28.6139, description='Latitude:', step=0.0001, style={'description_width': '100px'})
    longitude_widget = widgets.FloatText(value=77.2090, description='Longitude:', step=0.0001, style={'description_width': '100px'})
    buffer_widget = widgets.IntText(value=10, description='Buffer (km):', min=1, max=50, style={'description_width': '100px'})
    start_date_widget = widgets.Text(value=(datetime.now() - timedelta(days=30)).strftime('%Y-%m-%d'), description='Start Date:', style={'description_width': '100px'})
    interval_widget = widgets.IntSlider(value=5, min=1, max=60, description='Interval (min):', style={'description_width': '100px'})
    email_widget = widgets.Text(value=EMAIL_USER, description='Email:', style={'description_width': '100px'})

    status_output = widgets.Output()

    def update_status():
        with status_output:
            clear_output(wait=True)
            status = "🟢 ACTIVE" if monitoring_active else "🔴 STOPPED"
            display(HTML(f"""
            <div style="padding: 15px; background: #f5f5f5; border-radius: 5px;">
            <h3>System Status: {status}</h3>
            <p><strong>Time:</strong> {CURRENT_TIME} UTC</p>
            <p><strong>User:</strong> {CURRENT_USER}</p>
            <p><strong>Last Analysis:</strong> {last_analysis_date or 'None'}</p>
            <p><strong>Detection Types:</strong> Deforestation, Water Loss/Gain, Urban Growth, Vegetation Growth</p>
            </div>
            """))

    def start_handler(_):
        try:
            start_monitoring(latitude_widget.value, longitude_widget.value, buffer_widget.value,
                           start_date_widget.value, email_widget.value, interval_widget.value)
            update_status()
        except Exception as e:
            print(f"❌ Start error: {e}")

    def stop_handler(_):
        stop_monitoring()
        update_status()

    def manual_handler(_):
        try:
            system = MonitoringSystem()
            system.run_monitoring_cycle(latitude_widget.value, longitude_widget.value, buffer_widget.value,
                                      start_date_widget.value, email_widget.value)
            update_status()
        except Exception as e:
            print(f"❌ Manual cycle error: {e}")

    def reset_handler(_):
        reset_monitoring_dates()
        update_status()

    # Buttons
    btn_start = widgets.Button(description='🚀 Start Monitoring', button_style='success')
    btn_stop = widgets.Button(description='🛑 Stop Monitoring', button_style='danger')
    btn_manual = widgets.Button(description='🔍 Manual Cycle', button_style='info')
    btn_reset = widgets.Button(description='🔄 Reset Dates', button_style='warning')
    btn_refresh = widgets.Button(description='📊 Refresh Status', button_style='primary')

    btn_start.on_click(start_handler)
    btn_stop.on_click(stop_handler)
    btn_manual.on_click(manual_handler)
    btn_reset.on_click(reset_handler)
    btn_refresh.on_click(lambda _: update_status())

    # Layout
    controls = widgets.VBox([
        widgets.HTML("<h3>📍 Location Settings</h3>"),
        latitude_widget, longitude_widget, buffer_widget,
        widgets.HTML("<h3>⚙️ Monitoring Settings</h3>"),
        start_date_widget, interval_widget, email_widget,
        widgets.HTML("<h3>🎮 Controls</h3>"),
        widgets.HBox([btn_start, btn_stop]),
        widgets.HBox([btn_manual, btn_reset]),
        widgets.HBox([btn_refresh])
    ])

    update_status()
    dashboard = widgets.HBox([controls, status_output])

    display(widgets.VBox([
        widgets.HTML("""
        <div style="background: #4a90e2; color: white; padding: 20px; text-align: center; border-radius: 10px;">
        <h1>🛰️ Complete Satellite Monitoring System</h1>
        <p>Fixed Date Tracking • Water Body Analysis • All Change Types</p>
        <p>Deforestation 🌳 | Water Changes 💧 | Urban Growth 🏘️ | Vegetation Growth 🌱</p>
        </div>
        """), dashboard
    ]))

# ---- MAIN EXECUTION ----
if __name__ == "__main__":
    print("🛰️ COMPLETE SATELLITE MONITORING SYSTEM")
    print("=" * 60)
    print(f"Version: Complete Fixed v1.0")
    print(f"User: {CURRENT_USER}")
    print(f"Time: {CURRENT_TIME} UTC")
    print("Features: ✅ Fixed Date Tracking ✅ Water Analysis ✅ All Change Types")
    print("=" * 60)

    try:
        if authenticate_gee():
            print("✅ Starting complete dashboard...")
            create_dashboard()
        else:
            print("❌ Authentication failed")
    except Exception as e:
        print(f"❌ System error: {e}")
        display(HTML(f"""
        <div style="background: #f8d7da; color: #721c24; padding: 20px; border-radius: 5px;">
        <h3>❌ System Error</h3>
        <p>Error: {e}</p>
        <p>Please check authentication and try again.</p>
        </div>
        """))

✅ All dependencies loaded
🛰️ COMPLETE SATELLITE MONITORING SYSTEM
Version: Complete Fixed v1.0
User: Sahnik0
Time: 2025-07-01 12:18:49 UTC
Features: ✅ Fixed Date Tracking ✅ Water Analysis ✅ All Change Types
✅ GEE authentication successful
✅ Starting complete dashboard...
Initializing Complete Dashboard...



=== MONITORING CYCLE #1 ===
Location: 28.6139°N, 77.2090°E
Analysis Period: 2025-06-02 to 2025-07-01


✅ DEFORESTATION detected: 282.55 hectares, confidence: 0.950
✅ Email sent: 🌳🔥 DEFORESTATION DETECTED - 282.55 hectares
📧 deforestation alert email sent
✅ WATER_GAIN detected: 6336.97 hectares, confidence: 0.950
✅ Email sent: 💧⬆️ WATER GAIN DETECTED - 6336.97 hectares
📧 water_gain alert email sent
✅ URBAN_GROWTH detected: 87.51 hectares, confidence: 0.950
✅ Email sent: 🏘️📈 URBAN GROWTH DETECTED - 87.51 hectares
📧 urban_growth alert email sent


✅ VEGETATION_GROWTH detected: 3538.43 hectares, confidence: 0.950
✅ Email sent: 🌱📈 VEGETATION GROWTH DETECTED - 3538.43 hectares
📧 vegetation_growth alert email sent
=== CYCLE #1 COMPLETE ===
